In [26]:
import os

import pandas as pd
import numpy as np
from sklearn.metrics import fbeta_score
from tqdm import tqdm

from features import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

In [27]:
DATA_DIR = os.path.join('..', 'data')
os.listdir(DATA_DIR)

['history.csv', 'submission.csv', 'employees.csv']

In [28]:
hist = pd.read_csv(os.path.join(DATA_DIR, 'history.csv'))
empls = pd.read_csv(os.path.join(DATA_DIR, 'employees.csv'))
sbmsn = pd.read_csv(os.path.join(DATA_DIR, 'submission.csv'))

In [29]:
df = hist.merge(empls)

In [30]:
def label_df(df, n_month=3):
    labels = []
    for emp in df.EmployeeID.unique():
        curr_emp = list(df[df.EmployeeID == emp]['DismissalDate'])
        len_emp = len(curr_emp)
        if pd.isnull(curr_emp[0]):
            labels += [0 for i in range(len_emp - n_month)] + [2 for i in range(n_month)]
        else:
            labels += [0 for i in range(len_emp - n_month)] + [1 for i in range(n_month)]
    return labels

In [31]:
lbls= label_df(df)
unique, counts = np.unique(lbls, return_counts=True)
dict(zip(unique, counts))

{0: 71647, 1: 3051, 2: 13068}

In [32]:
df['target'] = lbls

In [33]:
df = df[df.target != 2]
np.unique(df.target)

array([0, 1])

In [36]:
data = df.head(1000)

In [37]:
num_features = ['Utilization', 'HourVacation', 'HourMobileReserve', 'HourLockedReserve', 'BonusOneTime', 'APM', 
                'WageGross']
cat_features_num = ['DevCenterID', 'SBUID', 'PositionID', 'PositionLevel', 'IsTrainee', 'LanguageLevelID', 
                    'IsInternalProject', 'OnSite', 'CompetenceGroupID', 'FunctionalOfficeID', 'PaymentTypeId',
                   'MonthOnPosition', 'MonthOnSalary']
cat_features_txt = ['CustomerID', 'ProjectID']

In [55]:
df = data.head(100)
df.shape

(100, 178)

In [61]:
for num_feature in num_features:
    df[num_feature + '_mean'] = get_mean_value(df, num_feature)
    df[num_feature + '_std'] = get_std_value(df, num_feature)
    df[num_feature + '_min'] = get_min_value(df, num_feature)
    df[num_feature + '_max'] = get_max_value(df, num_feature)
    df[num_feature + '_nunique'] = get_nunique_values(df, num_feature)
    df[num_feature + '_nunique_frac'] = get_nunique_values_fraction(df, num_feature)
    df[num_feature + '_last_m_max'] = get_last_minus_max(df, num_feature)
    df[num_feature + '_last_m_min'] = get_last_minus_min(df, num_feature)
    df[num_feature + '_last_m_mean'] = get_last_minus_mean(df, num_feature)
    df[num_feature + '_time_since_lat_change'] = get_time_since_last_change(df, num_feature)
    
for cat_feature in cat_features_num:
    df[cat_feature + '_std'] = get_std_value(df, cat_feature)
    df[cat_feature + '_min'] = get_min_value(df, cat_feature)
    df[cat_feature + '_max'] = get_max_value(df, cat_feature)
    df[cat_feature + '_nunique'] = get_nunique_values(df, cat_feature)
    df[cat_feature + '_nunique_frac'] = get_nunique_values_fraction(df, cat_feature)
    df[cat_feature + '_time_since_lat_change'] = get_time_since_last_change(df, cat_feature)
    
# for cat_feature in cat_features_txt:
#     df[cat_feature] = df[cat_feature].fillna('NA',inplace =True)
#     df[cat_feature + '_nunique'] = get_nunique_values(df, cat_feature)
#     df[cat_feature + '_nunique_frac'] = get_nunique_values_fraction(df, cat_feature)
#     df[cat_feature + '_time_since_lat_change'] = get_time_since_last_change(df, cat_feature)

In [24]:
df['Date'] = list(pd.to_datetime(df['Date']))
df["HiringDate"] = list(pd.to_datetime(df['HiringDate']))
df["Month_in_copmany"] = df.apply(lambda row: (row['Date'].year - row['HiringDate'].year)*12 + (row['Date'].month -row['HiringDate'].month), axis=1)

df["month_of_record"] = list(pd.DatetimeIndex(df['Date']).month)

df["Month_on_employee"] = get_count_month_for_columns(df, "CustomerID")
df["ProjectID"] = df["ProjectID"].astype(str)
df["Month_on_project"] = get_count_month_for_columns(df, "ProjectID")

df["Count_change_salary"] = count_change_salary(df)
df["Count_month_with_bonuses"] = count_month_with_having_something(df, "HourVacation")
df["Count_month_with_vacantion"] = count_month_with_having_something(df, "BonusOneTime")


In [25]:
df

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,IsInternalProject,Utilization,HourVacation,HourMobileReserve,HourLockedReserve,OnSite,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary,HiringDate,DismissalDate,target,Utilization_mean,Utilization_std,Utilization_min,Utilization_max,Utilization_nunique,Utilization_nunique_frac,Utilization_last_m_max,Utilization_last_m_min,Utilization_last_m_mean,Utilization_time_since_lat_change,HourVacation_mean,HourVacation_std,HourVacation_min,HourVacation_max,HourVacation_nunique,HourVacation_nunique_frac,HourVacation_last_m_max,HourVacation_last_m_min,HourVacation_last_m_mean,HourVacation_time_since_lat_change,HourMobileReserve_mean,HourMobileReserve_std,HourMobileReserve_min,HourMobileReserve_max,HourMobileReserve_nunique,HourMobileReserve_nunique_frac,HourMobileReserve_last_m_max,HourMobileReserve_last_m_min,HourMobileReserve_last_m_mean,HourMobileReserve_time_since_lat_change,HourLockedReserve_mean,HourLockedReserve_std,HourLockedReserve_min,HourLockedReserve_max,HourLockedReserve_nunique,HourLockedReserve_nunique_frac,HourLockedReserve_last_m_max,HourLockedReserve_last_m_min,HourLockedReserve_last_m_mean,HourLockedReserve_time_since_lat_change,BonusOneTime_mean,BonusOneTime_std,BonusOneTime_min,BonusOneTime_max,BonusOneTime_nunique,BonusOneTime_nunique_frac,BonusOneTime_last_m_max,BonusOneTime_last_m_min,BonusOneTime_last_m_mean,BonusOneTime_time_since_lat_change,APM_mean,APM_std,APM_min,APM_max,APM_nunique,APM_nunique_frac,APM_last_m_max,APM_last_m_min,APM_last_m_mean,APM_time_since_lat_change,WageGross_mean,WageGross_std,WageGross_min,WageGross_max,WageGross_nunique,WageGross_nunique_frac,WageGross_last_m_max,WageGross_last_m_min,WageGross_last_m_mean,WageGross_time_since_lat_change,DevCenterID_std,DevCenterID_min,DevCenterID_max,DevCenterID_nunique,DevCenterID_nunique_frac,DevCenterID_time_since_lat_change,SBUID_std,SBUID_min,SBUID_max,SBUID_nunique,SBUID_nunique_frac,SBUID_time_since_lat_change,PositionID_std,PositionID_min,PositionID_max,PositionID_nunique,PositionID_nunique_frac,PositionID_time_since_lat_change,PositionLevel_std,PositionLevel_min,PositionLevel_max,PositionLevel_nunique,PositionLevel_nunique_frac,PositionLevel_time_since_lat_change,IsTrainee_std,IsTrainee_min,IsTrainee_max,IsTrainee_nunique,IsTrainee_nunique_frac,IsTrainee_time_since_lat_change,LanguageLevelID_std,LanguageLevelID_min,LanguageLevelID_max,LanguageLevelID_nunique,LanguageLevelID_nunique_frac,LanguageLevelID_time_since_lat_change,IsInternalProject_std,IsInternalProject_min,IsInternalProject_max,IsInternalProject_nunique,IsInternalProject_nunique_frac,IsInternalProject_time_since_lat_change,OnSite_std,OnSite_min,OnSite_max,OnSite_nunique,OnSite_nunique_frac,OnSite_time_since_lat_change,CompetenceGroupID_std,CompetenceGroupID_min,CompetenceGroupID_max,CompetenceGroupID_nunique,CompetenceGroupID_nunique_frac,CompetenceGroupID_time_since_lat_change,FunctionalOfficeID_std,FunctionalOfficeID_min,FunctionalOfficeID_max,FunctionalOfficeID_nunique,FunctionalOfficeID_nunique_frac,FunctionalOfficeID_time_since_lat_change,PaymentTypeId_std,PaymentTypeId_min,PaymentTypeId_max,PaymentTypeId_nunique,PaymentTypeId_nunique_frac,PaymentTypeId_time_since_lat_change,MonthOnPosition_std,MonthOnPosition_min,MonthOnPosition_max,MonthOnPosition_nunique,MonthOnPosition_nunique_frac,MonthOnPosition_time_since_lat_change,MonthOnSalary_std,MonthOnSalary_min,MonthOnSalary_max,MonthOnSalary_nunique,MonthOnSalary_nunique_frac,MonthOnSalary_time_since_lat_change,CustomerID_nunique,CustomerID_nunique_frac,CustomerID_time_since_lat_change,ProjectID_nunique,ProjectID_nunique_frac,ProjectID_time_since_lat_change,Month_in_copmany,month_of_record,Month_on_employee,Month_on_project,Count_change_salary,Count_month_with_bonuses,Count_month_with_vacantion
0,00116D71-E87D-4B64-A566-1F29B2A798A8,2017-07-01,3,292,70,2,0,11,893EA22F-

In [ ]:
# fbeta_score(y_true, y_pred, beta=1.7)

In [ ]:
list(pd.DatetimeIndex(df['Date']).month)

In [62]:
df_csv = pd.read_csv('../df.csv')

In [66]:
df_csv

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,IsInternalProject,Utilization,HourVacation,HourMobileReserve,HourLockedReserve,OnSite,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary,HiringDate,DismissalDate,target,Utilization_mean,Utilization_std,Utilization_min,Utilization_max,Utilization_nunique,Utilization_nunique_frac,Utilization_last_m_max,Utilization_last_m_min,Utilization_last_m_mean,Utilization_time_since_lat_change,HourVacation_mean,HourVacation_std,HourVacation_min,HourVacation_max,HourVacation_nunique,HourVacation_nunique_frac,HourVacation_last_m_max,HourVacation_last_m_min,HourVacation_last_m_mean,HourVacation_time_since_lat_change,HourMobileReserve_mean,HourMobileReserve_std,HourMobileReserve_min,HourMobileReserve_max,HourMobileReserve_nunique,HourMobileReserve_nunique_frac,HourMobileReserve_last_m_max,HourMobileReserve_last_m_min,HourMobileReserve_last_m_mean,HourMobileReserve_time_since_lat_change,HourLockedReserve_mean,HourLockedReserve_std,HourLockedReserve_min,HourLockedReserve_max,HourLockedReserve_nunique,HourLockedReserve_nunique_frac,HourLockedReserve_last_m_max,HourLockedReserve_last_m_min,HourLockedReserve_last_m_mean,HourLockedReserve_time_since_lat_change,BonusOneTime_mean,BonusOneTime_std,BonusOneTime_min,BonusOneTime_max,BonusOneTime_nunique,BonusOneTime_nunique_frac,BonusOneTime_last_m_max,BonusOneTime_last_m_min,BonusOneTime_last_m_mean,BonusOneTime_time_since_lat_change,APM_mean,APM_std,APM_min,APM_max,APM_nunique,APM_nunique_frac,APM_last_m_max,APM_last_m_min,APM_last_m_mean,APM_time_since_lat_change,WageGross_mean,WageGross_std,WageGross_min,WageGross_max,WageGross_nunique,WageGross_nunique_frac,WageGross_last_m_max,WageGross_last_m_min,WageGross_last_m_mean,WageGross_time_since_lat_change,DevCenterID_std,DevCenterID_min,DevCenterID_max,DevCenterID_nunique,DevCenterID_nunique_frac,DevCenterID_time_since_lat_change,SBUID_std,SBUID_min,SBUID_max,SBUID_nunique,SBUID_nunique_frac,SBUID_time_since_lat_change,PositionID_std,PositionID_min,PositionID_max,PositionID_nunique,PositionID_nunique_frac,PositionID_time_since_lat_change,PositionLevel_std,PositionLevel_min,PositionLevel_max,PositionLevel_nunique,PositionLevel_nunique_frac,PositionLevel_time_since_lat_change,IsTrainee_std,IsTrainee_min,IsTrainee_max,IsTrainee_nunique,IsTrainee_nunique_frac,IsTrainee_time_since_lat_change,LanguageLevelID_std,LanguageLevelID_min,LanguageLevelID_max,LanguageLevelID_nunique,LanguageLevelID_nunique_frac,LanguageLevelID_time_since_lat_change,IsInternalProject_std,IsInternalProject_min,IsInternalProject_max,IsInternalProject_nunique,IsInternalProject_nunique_frac,IsInternalProject_time_since_lat_change,OnSite_std,OnSite_min,OnSite_max,OnSite_nunique,OnSite_nunique_frac,OnSite_time_since_lat_change,CompetenceGroupID_std,CompetenceGroupID_min,CompetenceGroupID_max,CompetenceGroupID_nunique,CompetenceGroupID_nunique_frac,CompetenceGroupID_time_since_lat_change,FunctionalOfficeID_std,FunctionalOfficeID_min,FunctionalOfficeID_max,FunctionalOfficeID_nunique,FunctionalOfficeID_nunique_frac,FunctionalOfficeID_time_since_lat_change,PaymentTypeId_std,PaymentTypeId_min,PaymentTypeId_max,PaymentTypeId_nunique,PaymentTypeId_nunique_frac,PaymentTypeId_time_since_lat_change,MonthOnPosition_std,MonthOnPosition_min,MonthOnPosition_max,MonthOnPosition_nunique,MonthOnPosition_nunique_frac,MonthOnPosition_time_since_lat_change,MonthOnSalary_std,MonthOnSalary_min,MonthOnSalary_max,MonthOnSalary_nunique,MonthOnSalary_nunique_frac,MonthOnSalary_time_since_lat_change,Month_in_copmany,month_of_record,Month_on_employee,Month_on_project,Count_change_salary,Count_month_with_bonuses,Count_month_with_vacantion
0,00116D71-E87D-4B64-A566-1F29B2A798A8,2017-07-01,3,292,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.7619,40,0,0,0,15,1,9,0,39,0.887446,1,1,2013-06-01,NaN,0,0.917570,0.145803,0.5000